In [ ]:
import dotenv
import os

from genai.model import GenerateParams
from genai import Credentials
from genai.extensions.langchain import LangChainInterface

In [ ]:
dotenv.load_dotenv()

api_key = os.getenv("GENAI_KEY",None)
api_endpoint = os.getenv("GENAI_API",None)

creds = Credentials(api_key,api_endpoint)

params = GenerateParams(
    decoding_method="greedy",
    max_new_tokens=300,
    min_new_tokens=15,
    repetition_penalty=2,
)

In [ ]:
llm = LangChainInterface(model="ibm/granite-3b-code-v1",credentials=creds,params=params)

In [ ]:
!pip install openai tiktoken chromadb langchain

In [ ]:
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser

In [ ]:
loader = GenericLoader.from_filesystem(
    "../../samples/",
    glob="**/*",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500)
)
documents = loader.load()
len(documents)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
python_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON, 
                                                               chunk_size=2000, 
                                                               chunk_overlap=200)
texts = python_splitter.split_documents(documents)
len(texts)

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceInstructEmbeddings
db = Chroma.from_documents(texts, HuggingFaceInstructEmbeddings())
retriever = db.as_retriever(
    search_type="mmr", # Also test "similarity"
    search_kwargs={"k": 8},
)

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain

memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [ ]:
question = "How can I initialize a ReAct agent?"
result = qa(question)
result['answer']

In [ ]:
questions = [
    "What is the class hierarchy?",
    "What classes are derived from the Chain class?",
    "What one improvement do you propose in code in relation to the class herarchy for the Chain class?",
]

for question in questions:
    result = qa(question)
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")